In [2]:
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve import internal
import netgen.gui
#import matplotlib.pyplot as plt
#%matplotlib inline

#internal.visoptions.autoscale = False

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
print(mesh.GetBoundaries())
Draw(mesh)

('bottom', 'right', 'top', 'left')


In [3]:
for bc in mesh.GetBoundaries():
    print(bc, "is top or right?:", bc=="right" or bc=="top")

bottom is top or right?: False
right is top or right?: True
top is top or right?: True
left is top or right?: False


In [4]:
Dir = "bottom"
Neu = "left"
Rob = "right|top"
display([bc for bc in mesh.GetBoundaries()])
display([y if (bc=="top" or bc=="right") else 0 for bc in mesh.GetBoundaries()])

['bottom', 'right', 'top', 'left']

[0,
 0]

In [5]:
f = 1
# (inhom) Dirichlet BC
ud = CoefficientFunction([x if bc=="bottom" else 0 for bc in mesh.GetBoundaries()])

# Not we set the coefficient functions for the Neumann and Robin BC..note the y --> CoefficientFunction defined by NGSolve
# Both Neumann and Robin depend on g
gRN = CoefficientFunction([y if bc!="bottom" else 0 for bc in mesh.GetBoundaries()])
gN = CoefficientFunction([y if bc=="left" else 0 for bc in mesh.GetBoundaries()])
gR = CoefficientFunction([y if (bc=="top" or bc=="right") else 0 for bc in mesh.GetBoundaries()])
# Robin boundary at top and right
alpha = 5
R_alpha = CoefficientFunction([alpha if (bc=="right" or  bc=="top") else 0 for bc in mesh.GetBoundaries()])
R_dalpha = CoefficientFunction([1/alpha if (bc=="right" or  bc=="top") else 0 for bc in mesh.GetBoundaries()])

In [6]:
runtimes = {"primal": -1, "mixed": -1}

Mathematical framework formulation:

$\int_{\Omega}\nabla u \nabla v -5\int_{\Gamma_R}uv = \int_{\Omega}v+\int_{\Gamma_N, \Gamma_R}yv$

For NGSolve:

*$\int_{\Omega}\nabla u \nabla v - R_{alpha} \int_{\Gamma_R}uv = \int_{\Omega}v+\int_{\Gamma_N, \Gamma_R} g\ v$*

In [7]:
from time import time_ns


def primal(mesh, order=2, draw=False, rt=False):
    FESp = H1(mesh, order=order, dirichlet="bottom")
    up, vp = FESp.TnT()

    # Forms
    ap = BilinearForm(FESp)
    ap += grad(up)*grad(vp)*dx  - R_alpha*up*vp*ds #(definedon=mesh.Boundaries("right|top"))

    fp = LinearForm(FESp)
    fp += f*vp*dx + gN*vp*ds #(definedon=mesh.Boundaries("left"))

    with TaskManager():
        ap.Assemble()
        fp.Assemble()

    # Gridfunction
    gfup = GridFunction(FESp, "u_primal")
    gfup.Set(ud, BND) # BND =^ definedon=mesh.Boundaries("bottom|right|top|left")

    # Need to take of the Dirichlet BC
    with TaskManager():
        start_p = time_ns()
        # Homogenyze the system... "do-it-yoursolve" version
        # rhs = fp.vec.CreateVector()
        # rhs.data = fp.vec - ap.mat * gfup.vec
        # gfup.vec.data += ap.mat.Inverse(freedofs=FESp.FreeDofs()) * rhs
        # Should be same as
        solvers.BVP(bf=ap, lf=fp, gf=gfup)
        if rt:
            delta_p = (time_ns() - start_p) * 1e-9
            print(f"Runtime primal: {delta_p}s")
            runtimes["primal"] = round(delta_p, 3)

    flux = CoefficientFunction(grad(gfup)) #R_alpha *

    if draw:
        Draw (gfup)
        Draw (flux, mesh, "flux_primal")
        internal.visoptions.autoscale = False
    return gfup, flux

# Mixed formulation: similar to Example 4.1


In [8]:
def mixed(mesh, order_flux=2, draw=False, rt=False):
    # Setup the mixed method
    V = HDiv(mesh, order=order_flux, dirichlet="left") # "Neumann is the new Dirichlet" 
    Q = L2(mesh, order=order_flux-1) #, dirichlet="bottom"

    FESm = FESpace([V,Q]) # Compound space
    TnTm = FESm.TnT()
    (sigma, u), (tau, v) = TnTm 

    # Setup the special function "normal"
    #print("mesh.dim:", mesh.dim)
    normal = specialcf.normal(mesh.dim)
    #print(normal)

    # am += (sigma*tau - div(sigma)*v + div(tau)*u)*dx
    # fm += source*v*dx + ud*(tau.Trace()*normal)*ds + R*u*ds

    # Set up forms again
    am = BilinearForm(FESm)
    #      a(sig,tau)+ b(sig,v)     + b(tau,u)       + bilinear Robin-Term
    am += (sigma*tau + div(sigma)*v + div(tau)*u)*dx + (R_dalpha*(tau.Trace()*normal)*(sigma.Trace()*normal))*ds #(definedon=mesh.Boundaries("top|right"))
    am.Assemble()

    fm = LinearForm(FESm)
    #     source  + dirichlet                  + linear Robin-term
    fm += +f*v*dx + ud*(tau.Trace()*normal)*ds - gR*R_dalpha*tau.Trace()*normal*ds #(definedon=mesh.Boundaries("top|right"))
    # ud*(tau.Trace()*normal)*ds(definedon=mesh.Boundaries("bottom"))
    # + gR*(tau.Trace()*normal)*ds
    fm.Assemble()

    # Now on to the actual problem
    gfm = GridFunction(FESm, name="gf_mixed")
    gfsigma, gfum = gfm.components
    # Remember g from before? :D
    # It was defined as scalar valued --> "need to bring it into the vector world"
    gfsigma.Set(normal*gN, BND) #definedon=mesh.Boundaries("left")) # definedon=mesh.Boundaries("left") / "bottom|right|top|left"
    #gfum.Set(ud)

    with TaskManager():
        start_m = time_ns()
        fm.vec.data -= am.mat * gfm.vec
        gfm.vec.data += am.mat.Inverse(freedofs=FESm.FreeDofs(), inverse="umfpack") * fm.vec
        #solvers.BVP(bf=am, lf=fm, gf=gfm)
        delta_m = (time_ns() - start_m) * 1e-9
        if rt:
            print(f"Runtime mixed: {delta_m}s")
            runtimes["mixed"] = round(delta_m, 3)

    if draw:
        Draw (gfsigma, mesh, "flux_mixed")
        Draw (gfum, mesh, "u_mixed")
        internal.visoptions.autoscale = False
    return gfm

In [9]:
hs = [0.5, 0.2, 0.1, 0.08, 0.05, 0.025, 0.01]
def compare_PvM(h_list=hs):
    errors_flux = []
    errors_u = []
    for h in h_list:
        mesh = Mesh(unit_square.GenerateMesh(maxh=h))
        print("h:", h)

        # Solve the problems
        order = 2
        gfup, flux = primal(mesh, order=order, draw=False)
        gfm = mixed(mesh, order_flux=order+1, draw=False)
        gfsigma, gfum = gfm.components

        # Errors
        err_u = sqrt(Integrate( (gfup-gfum)*(gfup-gfum), mesh))
        errors_u.append(err_u)
        print ("err_u:", err_u)
        flux_diff = grad(gfup) - gfsigma
        err_flux = sqrt(Integrate(flux_diff*flux_diff, mesh))
        errors_flux.append(err_flux)
        print ("err_flux:", err_flux)
        
        Draw (gfup, mesh, "u_primal")
        Draw (flux, mesh, "flux_primal")
        Draw (gfum, mesh, "u_mixed")
        Draw (gfsigma, mesh, "flux_mixed")
        internal.visoptions.autoscale = False
        stop = input("Stop? (y/n): ")
        if stop == 'y':
            break
    return errors_u, errors_flux

In [10]:
hs = [0.5, 0.2, 0.1, 0.08, 0.05, 0.025, 0.01]
errors_u, errors_flux = compare_PvM(h_list=hs)

hs_used = hs[0:int(len(errors_u))]

h: 0.5
err_u: 0.7348043890578521
err_flux: 4.742834563452252


Stop? (y/n):  y


In [ ]:
plt.figure(figsize=(10,8))
plt.plot(hs_used, errors_u, label="err(u)")
plt.plot(hs_used, errors_flux, label="err(flux)")
plt.grid()
plt.legend()
plt.show()

In [ ]:
# "The" gridfunction...compound space of 2 spaces --> 2 components
gfmTest = GridFunction(FESm, name="gf_mixedTest")
gfsigmaTest, gfumTest = gfmTest.components

# Let's draw something just to see how it behaves
# gfsigmaTest.Set(CoefficientFunction( (sin(10*x), sin(10*y)) ))

gfsigma.Set(g*normal, definedon=BND)
gfumTest.Set (x)

# Draw (gfsigmaTest, mesh, "sinx_siny")
Draw (gfsigmaTest, mesh, "g*normal")
Draw (gfumTest, mesh, "x")